In [132]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from unidecode import unidecode
import re
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'scripts')))
from extract_grammys import create_db_connection, read_query
from extract_spotify import extract_spotify_data


In [133]:
df_spotify= extract_spotify_data("../data/spotify_dataset.csv")

Dataset de Spotify cargado correctamente desde ../data/spotify_dataset.csv
Shape: (114000, 21)


In [134]:
data_grammys = "SELECT * FROM db_grammys.grammys"

In [135]:
connection = create_db_connection("localhost", "root", "annie", "db_grammys") # Connect to the Database
results = read_query(connection, data_grammys)

MySQL Database connection successful


In [136]:
df_grammys= pd.DataFrame(results, columns=['id','year','title','published_at','updated_at', 'category', 'nominee','artist', 'workers','img','winner'])
df_grammys.head()

,id,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,1
1,2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,1
2,3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,1
3,4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,1
4,5,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,1


In [139]:
def clean_text(text):
    """Limpia y normaliza texto para comparación"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = unidecode(text)  # Elimina acentos
    text = re.sub(r'[^\w\s]', '', text)  # Elimina puntuación
    text = re.sub(r'\s+', ' ', text).strip()  # Normaliza espacios
    return text

In [140]:
def transform_grammys(df_grammys):
    df = df_grammys.copy()
    
    # fill img and nominee null values 
    df['img'] = df['img'].fillna("no image available")
    df['nominee'] = df['nominee'].fillna("Unknown")

    #Categories where artist can be inferred from nominee
    artist_from_nominee_categories = [
        'Best New Artist',
        'Best Pop Solo Performance',
        'Best Pop Duo/Group Performance',
        'Best Traditional Pop Vocal Album',
        'Best Pop Vocal Album',
        'Best Dance Recording',
        'Best Dance/Electronic Album',
        'Best Contemporary Instrumental Album',
        'Best Rock Performance',
        'Best Metal Performance',
        'Best Rock Album',
        'Best Alternative Music Album',
        'Best R&B Performance',
        'Best Traditional R&B Performance',
        'Best R&B Album',
        'Best Rap Performance',
        'Best Rap Album',
        'Best Country Solo Performance',
        'Best Country Duo/Group Performance',
        'Best Country Album',
        'Best Jazz Vocal Album',
        'Best Jazz Instrumental Album',
        'Best Latin Pop Album',
        'Best Regional Mexican Music Album',
        'Best Gospel Performance/Song',
        'Best Contemporary Christian Music Album',
        'Best Reggae Album',
        'Best World Music Album',
        'Best Children\'s Album',
        'Best Comedy Album',
        'Best Musical Theater Album',
        'Best Compilation Soundtrack For Visual Media'
    ]
    
    # Categories where artist cannot be inferred from nominee
    technical_categories = [
        'Song Of The Year',  
        'Best Opera Recording',  
        'Best Album Notes',  
        'Best Country Song',  
        'Best Instrumental Composition',  
        'Best Historical Album',  
        'Best Chamber Music Performance',  
        'Best Instrumental Arrangement',  
        'Best Orchestral Performance',  
        'Best Classical Album',  
        'Best Rock Song',  
        'Best Rhythm & Blues Song',  
        'Best Recording Package',  
        'Best Choral Performance',  
        'Best Engineered Album, Classical',  
        'Producer Of The Year, Non-Classical',  
        'Producer Of The Year, Classical',  
        'Best Engineered Recording - Non-Classical',  
        'Best R&B Song'  
    ]
    
    mask_nominee = (df['artist'].isna()) & (df['category'].isin(artist_from_nominee_categories))
    df.loc[mask_nominee, 'artist'] = df.loc[mask_nominee, 'nominee']
    
    # For technical categories, fill with "Various Artists"
    df['artist'] = df['artist'].fillna("Varius Artists")

    #For workers column, fill NaN with "Unknown"
    df['workers'] = df['workers'].fillna("Unknown")

    # clean and convert date columns
    df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
    df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')
    
    # normalize category column 
    df['category'] = df['category'].str.strip()

    # split and clean artist names
    df['artist_list'] = df['artist'].str.split(',|&|feat\.|featuring', regex=True)
    df['artist_list'] = df['artist_list'].apply(
        lambda x: [clean_text(artist) for artist in x] if isinstance(x, list) else [clean_text(x)]
    )
    #convert winner column to boolean
    df['winner'] = df['winner'].astype(bool)

    return df

<>:84: SyntaxWarning: invalid escape sequence '\.'
<>:84: SyntaxWarning: invalid escape sequence '\.'
C:\Users\annie\AppData\Local\Temp\ipykernel_2016\2789234171.py:84: SyntaxWarning: invalid escape sequence '\.'
  df['artist_list'] = df['artist'].str.split(',|&|feat\.|featuring', regex=True)


In [141]:
df_grammys_clean = transform_grammys(df_grammys)

C:\Users\annie\AppData\Local\Temp\ipykernel_2016\2789234171.py:77: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
C:\Users\annie\AppData\Local\Temp\ipykernel_2016\2789234171.py:78: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')


In [159]:
df_grammys_clean.head()

,id,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner,artist_list
0,1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28-07:00,2020-05-19 05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True,[billie eilish]
1,2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28-07:00,2020-05-19 05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True,[bon iver]
2,3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28-07:00,2020-05-19 05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True,[ariana grande]
3,4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28-07:00,2020-05-19 05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True,[her]
4,5,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28-07:00,2020-05-19 05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True,[khalid]


In [145]:
df_grammys_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            4810 non-null   int64 
 1   year          4810 non-null   int64 
 2   title         4810 non-null   object
 3   published_at  4810 non-null   object
 4   updated_at    4810 non-null   object
 5   category      4810 non-null   object
 6   nominee       4810 non-null   object
 7   artist        4810 non-null   object
 8   workers       4810 non-null   object
 9   img           4810 non-null   object
 10  winner        4810 non-null   bool  
 11  artist_list   4810 non-null   object
dtypes: bool(1), int64(2), object(9)
memory usage: 418.2+ KB


In [152]:
def transform_spotify(df_spotify):
    df = df_spotify.copy()
    
    # delete Unnamed: 0 column
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    # rename columns for consistency
    df = df.rename(columns={
        'artists': 'artist',  # Singular para consistencia con Grammys
        'album_name': 'album',  # Más conciso
        'track_name': 'track',  # Más conciso
        'track_genre': 'genre'})
    
    # fill missing values
    df['artist'] = df['artist'].fillna("Unknown")
    df['album'] = df['album'].fillna("Unknown")
    df['track'] = df['track'].fillna("Unknown")

    # split and clean artist 
    df['artists_list'] = df['artist'].str.split(',|;', regex=True)
    df['artists_list'] = df['artists_list'].apply(
        lambda x: [clean_text(artist) for artist in x] if isinstance(x, list) else [clean_text(x)]
    )
    
    # convert duration from ms to min
    df['duration_min'] = df['duration_ms'] / 60000
    
    #crate popularity category
    df['popularity_category'] = pd.cut(
        df['popularity'],
        bins=[-np.inf, 35, 50, np.inf],  
        labels=['Low', 'Medium', 'High']
    )

    #normalize genre column
    df['genre'] = df['genre'].str.lower().str.strip()

    # remove duplicates based on track_id, keeping the first occurrence
    df = df.drop_duplicates(subset=['track_id'], keep='first')

    return df

In [153]:
df_spotify_clean = transform_spotify(df_spotify)
df_spotify_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89741 entries, 0 to 113999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   track_id             89741 non-null  object  
 1   artist               89741 non-null  object  
 2   album                89741 non-null  object  
 3   track                89741 non-null  object  
 4   popularity           89741 non-null  int64   
 5   duration_ms          89741 non-null  int64   
 6   explicit             89741 non-null  bool    
 7   danceability         89741 non-null  float64 
 8   energy               89741 non-null  float64 
 9   key                  89741 non-null  int64   
 10  loudness             89741 non-null  float64 
 11  mode                 89741 non-null  int64   
 12  speechiness          89741 non-null  float64 
 13  acousticness         89741 non-null  float64 
 14  instrumentalness     89741 non-null  float64 
 15  liveness             89